In [1]:
import random
import json
import time
import requests
import pandas as pd

In [21]:
pd.options.display.max_columns = None


def str_to_dict(x):
    return eval('{"' + x.replace(": ", '":"').replace('\n', '","') + '"}')


headers = str_to_dict("""accept: */*
accept-encoding: gzip, deflate, br
accept-language: zh-CN,zh;q=0.9
referer: https://space.bilibili.com/6574487/fans/follow
sec-fetch-dest: script
sec-fetch-mode: no-cors
sec-fetch-site: same-site
user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36""")


# 获取单个UP主的关注对象
def follows_spider(up_id):
    data = []
    page = 1
    # B站默认限制查看前100个关注者
    while page <= 5:
        time.sleep(random.randint(6, 12) * 0.1)
#         print(up_id, page)
        # 关注者api
        api_url = "http://api.bilibili.com/x/relation/followings?vmid=%s&pn=%s&ps=50&order=desc&jsonp=jsonp&callback=__jp5" % (
            up_id, page)
        response = requests.get(api_url, headers=headers)
        
        try:
            # 关注者
            follows = json.loads(response.text[6:-1])["data"]["list"]
        except Exception as e:
            print(e)
            print(json.loads(response.text[6:-1]))
            break

        data.extend(follows)
        page += 1
    return data


# BFS
# 广度优先搜索UP主的关注对象
def follow_tree_spider(seeds, deep=1):
    # 详细数据
    final_data = pd.DataFrame()
    # 已获取
    from_cache = set()
    # 待获取
    queue = set(seeds)
    
    try:
        for i in range(deep):
            print("正在获取第%级关注."%i)
            sub_data = pd.DataFrame()
            for seed in queue:
                sub_data = pd.DataFrame(follows_spider(str(int(float(seed)))))
                sub_data["from"] = seed
                final_data = final_data.append(sub_data)
            # 单层搜索结束进行缓存更新
            from_cache.update(queue)
            # 当前列表去除缓存数据
            # print(sub_data.columns)
            queue = set(final_data["mid"]) - from_cache
    except Exception as e:
        print(e)
    return final_data

In [22]:
headers

{'accept': '*/*',
 'accept-encoding': 'gzip, deflate, br',
 'accept-language': 'zh-CN,zh;q=0.9',
 'referer': 'https://space.bilibili.com/6574487/fans/follow',
 'sec-fetch-dest': 'script',
 'sec-fetch-mode': 'no-cors',
 'sec-fetch-site': 'same-site',
 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}

In [ ]:
seeds = ['6574487']
data = follow_tree_spider(seeds, 3)

In [25]:
len(data)

8966

In [41]:
data["mid"] = data["mid"].astype(int)
data["from"] = data["from"].astype(int)

In [45]:
len(data.mid.unique())

5551

In [46]:
len(data["from"].unique())

89

In [53]:
seeds2 = list(set(data["mid"]) - set(data["from"]))[:10]

In [ ]:
data2 = follow_tree_spider(seeds2, 2)

In [27]:
data.to_excel("../data/follows_deep3_self.xlsx",index=True)

In [64]:
data2.to_csv("../data/follows_deep3_self2.csv",encoding="utf_8_sig")

In [63]:
len(data2)

62277